In [2]:
import os
import csv
import statistics
import numpy

In [3]:
#Function to compare two groups
def compare_one(x):
    error = 0
    count = 0
    fp_b_25 = 0
    fp_b_25_45 = 0
    fp_b_45 = 0
    fp_w_25 = 0
    fp_w_25_45 = 0
    fp_w_45 = 0
    npv_b_25 = 0
    npv_b_25_45 = 0
    npv_b_45 = 0
    npv_w_25 = 0
    npv_w_25_45 = 0
    npv_w_45 = 0
    with open("propublica_full.csv", "r", encoding='utf-8') as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
            if row["race"] == "African-American":
                count += 1
                if int(row["decile_score"]) > x: 
                    if row["is_recid"] == "0":
                        error += 1
                        if row["age_cat"] == "Less than 25" : fp_b_25 += 1
                        if row["age_cat"] == "25 - 45" : fp_b_25_45 += 1
                        if row["age_cat"] == "Greater than 25" : fp_b_45 += 1
                if int(row["decile_score"]) < x: 
                    if row["is_recid"] == "0":
                        if row["age_cat"] == "Less than 25" : npv_b_25 += 1
                        if row["age_cat"] == "25 - 45" : npv_b_25_45 += 1
                        if row["age_cat"] == "Greater than 25" : npv_b_45 += 1
                    if row["is_recid"] == "1":
                        error+= 1
            if row["race"] == "Caucasian":
                count += 1
                if int(row["decile_score"]) > x: 
                    if row["is_recid"] == "0":
                        error += 1
                        if row["age_cat"] == "Less than 25" : fp_w_25 += 1
                        if row["age_cat"] == "25 - 45" : fp_w_25_45 += 1
                        if row["age_cat"] == "Greater than 25" : fp_w_45 += 1
                if int(row["decile_score"]) < x: 
                    if row["is_recid"] == "0":
                        if row["age_cat"] == "Less than 25" : npv_w_25 += 1
                        if row["age_cat"] == "25 - 45" : npv_w_25_45 += 1
                        if row["age_cat"] == "Greater than 25" : npv_w_45 += 1
                    if row["is_recid"] == "1":
                        error+= 1
        
    if fp_b_25 == 0 and npv_b_25 == 0 : npv_b_25 += 1
    if fp_b_25_45 == 0 and npv_b_25_45 == 0 : npv_b_25_45 += 1
    if fp_b_45 == 0 and npv_b_45 == 0 : npv_b_45 += 1
    if fp_w_25 == 0 and npv_w_25 == 0 : npv_w_25 += 1
    if fp_w_25_45 == 0 and npv_w_25_45 == 0 : npv_w_25_45 += 1
    if fp_w_45 == 0 and npv_w_45 == 0 : npv_w_45 += 1
    
        
    fp_rate_b_25 = 100*fp_b_25/(fp_b_25 + npv_b_25)
    fp_rate_b_25_45 = 100*fp_b_25_45/(fp_b_25_45 + npv_b_25_45)
    fp_rate_b_45 = 100*fp_b_25/(fp_b_45 + npv_b_45)
    fp_rate_w_25 = 100*fp_w_25/(fp_w_25 + npv_w_25)
    fp_rate_w_25_45 = 100*fp_w_25_45/(fp_w_25_45 + npv_w_25_45)
    fp_rate_w_45 = 100*fp_w_45/(fp_w_45 + npv_w_45)
    return [100*error/count, numpy.std([fp_rate_b_25, fp_rate_b_25_45, fp_rate_b_45,
                                        fp_rate_w_25, fp_rate_w_25_45, fp_rate_w_45]) ]



def compare_two(x,y, group1, group2, cat):
    error = 0
    false_positive_group1 = 0
    false_negative_group1 = 0
    ppv_group1 = 0
    npv_group1 = 0
    false_positive_group2 = 0
    false_negative_group2 = 0
    ppv_group2 = 0
    npv_group2 = 0
    count = 0 
    with open("propublica_full.csv", "r", encoding='utf-8') as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
            if row[cat] == group1: 
                count += 1
                if int(row["decile_score"]) > x: 
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_group1 += 1
                    elif row["is_recid"] == "1":
                        ppv_group1 += 1
                elif int(row["decile_score"]) < x: 
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_group1 += 1
                    elif row["is_recid"] == "0":
                        npv_group1 += 1
            if row[cat] == group2: 
                count += 1
                if int(row["decile_score"]) > y:    
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_group2 += 1
                    elif row["is_recid"] == "1":
                        ppv_group2 += 1
                elif int(row["decile_score"]) < y: 
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_group2 += 1
                    elif row["is_recid"] == "0":
                        npv_group2 += 1
    if false_positive_group1 == 0 and npv_group1 == 0:
        npv_group1 += 1
    if false_positive_group2 == 0 and npv_group2 == 0:
        npv_group2 += 1
    return [100*error/count, 100*abs((false_positive_group1/(false_positive_group1 + npv_group1)
                                      - false_positive_group2/(false_positive_group2 + npv_group2)))]

#Function to compare three groups
def compare_three(x,y,z, group1, group2, group3, cat):
    error = 0
    false_positive_group1 = 0
    false_negative_group1 = 0
    ppv_group1 = 0
    npv_group1 = 0
    false_positive_group2 = 0
    false_negative_group2 = 0
    ppv_group2 = 0
    npv_group2 = 0
    false_positive_group3 = 0
    false_negative_group3 = 0
    ppv_group3 = 0
    npv_group3 = 0
    false_positive_white = 0
    false_negative_white = 0
    ppv_group_white = 0
    npv_group_white = 0
    false_positive_black = 0
    false_negative_black = 0
    ppv_group_black = 0
    npv_group_black = 0
    count = 0 
    
    with open("propublica_full.csv", "r", encoding='utf-8') as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
            if row[cat] == group1: 
                count += 1
                if int(row["decile_score"]) > x: 
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_group1 += 1
                        if row["race"] == "African-American":
                            false_positive_black += 1
                        elif row["race"] == "Caucasian":
                            false_positive_white += 1
                    elif row["is_recid"] == "1":
                        ppv_group1 += 1
                        if row["race"] == "African-American":
                            ppv_group_black += 1
                        elif row["race"] == "Caucasian":
                            ppv_group_white += 1
                elif int(row["decile_score"]) < x: 
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_group1 += 1
                        if row["race"] == "African-American":
                            false_negative_black += 1
                        elif row["race"] == "Caucasian":
                            false_negative_white += 1
                    elif row["is_recid"] == "0":
                        npv_group1 += 1
                        if row["race"] == "African-American":
                            npv_group_black += 1
                        elif row["race"] == "Caucasian":
                            npv_group_white += 1
            if row[cat] == group2: 
                count += 1
                if int(row["decile_score"]) > y:    
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_group2 += 1
                        if row["race"] == "African-American":
                            false_positive_black += 1
                        elif row["race"] == "Caucasian":
                            false_positive_white += 1
                    elif row["is_recid"] == "1":
                        ppv_group2 += 1
                        if row["race"] == "African-American":
                            ppv_group_black += 1
                        elif row["race"] == "Caucasian":
                            ppv_group_white += 1
                elif int(row["decile_score"]) < y: 
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_group2 += 1
                        if row["race"] == "African-American":
                            false_negative_black += 1
                        elif row["race"] == "Caucasian":
                            false_negative_white += 1
                    elif row["is_recid"] == "0":
                        npv_group2 += 1
                        if row["race"] == "African-American":
                            npv_group_black += 1
                        elif row["race"] == "Caucasian":
                            npv_group_white += 1
            if row[cat] == group3: 
                count += 1
                if int(row["decile_score"]) > z:    
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_group3 += 1
                        if row["race"] == "African-American":
                            false_positive_black += 1
                        elif row["race"] == "Caucasian":
                            false_positive_white += 1
                    elif row["is_recid"] == "1":
                        ppv_group3 += 1
                        if row["race"] == "African-American":
                            ppv_group_black += 1
                        elif row["race"] == "Caucasian":
                            ppv_group_white += 1
                elif int(row["decile_score"]) < z: 
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_group3 += 1
                        if row["race"] == "African-American":
                            false_negative_black += 1
                        elif row["race"] == "Caucasian":
                            false_negative_white += 1
                    elif row["is_recid"] == "0":
                        npv_group3 += 1
                        if row["race"] == "African-American":
                            npv_group_black += 1
                        elif row["race"] == "Caucasian":
                            npv_group_white += 1
    if false_positive_group1 == 0 and npv_group1 == 0:
        npv_group1 += 1
    if false_positive_group2 == 0 and npv_group2 == 0:
        npv_group2 += 1
    if false_positive_group3 == 0 and npv_group3 == 0:
        npv_group3 += 1
    if false_positive_white == 0 and npv_group_white == 0:
        npv_group_white += 1
    if false_positive_black == 0 and npv_group_black == 0:
        npv_group_black += 1
        
    fp1 = 100*abs((false_positive_group1/(false_positive_group1 + npv_group1)
                                      - false_positive_group2/(false_positive_group2 + npv_group2)))
    fp2 = 100*abs((false_positive_group2/(false_positive_group2 + npv_group2)
                                      - false_positive_group3/(false_positive_group3 + npv_group3)))
    fp3 = 100*abs((false_positive_group1/(false_positive_group1 + npv_group1)
                                      - false_positive_group3/(false_positive_group3 + npv_group3)))
    fp_bw = 100*abs((false_positive_white/(false_positive_white + npv_group_white)
                                      - false_positive_black/(false_positive_black + npv_group_black)))
    return [100*error/count, numpy.std([fp1, fp2, fp3]), fp_bw]

In [4]:
def errorFuncTwo(x, y, group1, group2, cat): 
 for i in range(11): 
     for j in range(11): 
        returned = compare_two(i, j, group1, group2, cat)
        x.append(returned[0])
        y.append(returned[1])
def errorFuncThree(x, y, z, group1, group2, group3, cat):
    for i in range(11):
        for j in range(11):
            for k in range(11):
                returned = compare_three(i, j, k, group1, group2, group3, cat)
                x.append(returned[0])
                y.append(returned[1])
                z.append(returned[2])
                
def errorFuncOne(x,y):
    for i in range(11):
        return_list = compare_one(i)
        x.append(return_list[0])
        y.append(return_list[1])

In [5]:
#Filling in the low-mid arrays
low_mid_error = []
low_mid_fp = []
arr1 = errorFuncTwo(low_mid_error, low_mid_fp, "Less than 25", "25 - 45", "age_cat")
#Filling  in the mid-hi arrays
mid_hi_error = []
mid_hi_fp = []
arr2 = errorFuncTwo(mid_hi_error, mid_hi_fp, "25 - 45", "Greater than 45", "age_cat")
#Filling  in the low-hi arrays
low_hi_error = []
low_hi_fp = []
arr3 = errorFuncTwo(low_hi_error, low_hi_fp, "Less than 25", "Greater than 45", "age_cat")

#Filling in Black White array
bw_error = []
bw_fp = []
arr4 = errorFuncTwo(bw_error, bw_fp, "African-American", "Caucasian", "race")

lowest_fp_error = []
lowest_fp = []
for i in range(len(low_hi_fp)):
    mini_fp = min(low_mid_fp[i], mid_hi_fp[i], low_hi_fp[i])
    if mini_fp == low_mid_fp[i]:
        lowest_fp_error.append(low_mid_error[i])
        lowest_fp.append(low_mid_fp[i])
    elif mini_fp == mid_hi_fp[i]:
        lowest_fp_error.append(mid_hi_error[i])
        lowest_fp.append(mid_hi_fp[i])
    else:
        lowest_fp_error.append(low_hi_error[i])
        lowest_fp.append(low_hi_fp[i])
        
lowest_error = []
lowest_error_fp = []
for i in range(len(low_hi_fp)):
    mini_err = min(low_mid_error[i], mid_hi_error[i], low_hi_error[i])
    if mini_err == low_mid_error[i]:
        lowest_error.append(low_mid_error[i])
        lowest_error_fp.append(low_mid_fp[i])
    elif mini_err == mid_hi_error[i]:
        lowest_error.append(mid_hi_error[i])
        lowest_error_fp.append(mid_hi_fp[i])
    else:
        lowest_error.append(low_hi_error[i])
        lowest_error_fp.append(low_hi_fp[i])
    
# avg_error = []
# std = []
# for i in range(len(low_hi_fp)):
#     avg_error.append(statistics.mean([low_mid_error[i],mid_hi_error[i],low_hi_error[i]]))
#     std.append(numpy.std([low_mid_error[i],mid_hi_error[i],low_hi_error[i]]))
    
total_error = []
std = []
fp_bw_arr = []
array = errorFuncThree(total_error, std, fp_bw_arr, "Less than 25", "25 - 45", "Greater than 45", "age_cat")

In [6]:
error_new = []
fp_std_new = []
array = errorFuncOne(error_new, fp_std_new)

In [7]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, gridplot

# output to static HTML file
output_file("line.html")

data = {'low_mid_error': low_mid_error,
        'low_mid_fp': low_mid_fp,
        'mid_hi_error': mid_hi_error,
        'mid_hi_fp': mid_hi_fp,
        'low_hi_error': low_hi_error,
        'low_hi_fp': low_hi_fp,
        'bw_error': bw_error,
        'bw_fp': bw_fp}

data2 = {'total_error': total_error,
         'std': std,
         'bw_fp_error': fp_bw_arr}


source = ColumnDataSource(data=data)

source2 = ColumnDataSource(data=data2)

#P1 - low mid
p1 = figure(title= "Less than 25 and 25 - 45", plot_width=600, plot_height=600, tooltips=[("Error", "@low_mid_error%"), ("False Positive Rate", "@low_mid_fp%")])
p1.circle(x='low_mid_error', y='low_mid_fp', size=5, source=source, color="navy", alpha=0.5)
p1.xaxis.axis_label = "Error"
p1.yaxis.axis_label = "False Positive Rate"

#P2 - mid high
p2 = figure(title= "25 - 45 and Greater than 45", plot_width=600, plot_height=600, tooltips=[("Error", "@mid_hi_error%"), ("False Positive Rate", "@mid_hi_fp%")])
p2.circle(x='mid_hi_error', y='mid_hi_fp', size=5, source=source, color="navy", alpha=0.5)
p2.xaxis.axis_label = "Error"
p2.yaxis.axis_label = "False Positive Rate"

#P3 - low high
p3 = figure(title= "Less than 25 and Greater than 45", plot_width=600, plot_height=600, tooltips=[("Error", "@low_hi_error%"), ("False Positive Rate", "@low_hi_fp%")])
p3.circle(x='low_hi_error', y='low_hi_fp', size=5, source=source, color="navy", alpha=0.5)
p3.xaxis.axis_label = "Error"
p3.yaxis.axis_label = "False Positive Rate"

#P4 - avg error vs std dev
p4 = figure(title= "Age error and Standard Deviation", plot_width=600, plot_height=600, tooltips=[("Total Age Error", "@total_error%"), ("Standard Deviation", "@std")])
p4.circle(x='total_error', y='std', size=5, source=source2, color="navy", alpha=0.5)
p4.xaxis.axis_label = "Age Error"
p4.yaxis.axis_label = "Standard Deviation between all three false positive age pairings"

p5 = figure(title= "Black and White", plot_width=600, plot_height=600, tooltips=[("Error", "@bw_error%"), ("False Positive Rate", "@bw_fp%")])
p5.circle(x='bw_error', y='bw_fp', size=5, source=source, color="navy", alpha=0.5)
p5.xaxis.axis_label = "Error"
p5.yaxis.axis_label = "False Positive Rate between Black and White"

p6 = figure(title= "Black and White False Positive and Standard Deviation between Age Categories", plot_width=600, plot_height=600, tooltips=[("Standard Deviation", "@std"), ("False Positive Rate", "@bw_fp_error%"), ("Error", "@total_error%")])
p6.circle(x='std', y='bw_fp_error', size=5, source=source2, color="navy", alpha=0.5)
p6.xaxis.axis_label = "Standard Deviation between all three false positive age pairings"
p6.yaxis.axis_label = "False Positive Rate between Black and White"
# show the results
output_file("index.html")
show(gridplot([[p1,p2],[p3,p5],[p4,p6]]))


In [1]:
for i in range(11):
    print(i)

0
1
2
3
4
5
6
7
8
9
10


In [6]:
x= 0 
if x < 1 : x += 1 